# 1. Importing relevant packages for neural networks

In [ ]:
import os 
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Conv2D, MaxPooling2D, concatenate, Reshape, Conv2DTranspose, LeakyReLU, BatchNormalization, Activation, UpSampling2D 
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import models 
from tensorflow.keras import backend as K
from keras.callbacks import History 
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 2. Assigning a seed

In [ ]:
seed = 0 
random.seed = seed
np.random.seed = seed

# 3. Defining variables

In [ ]:
SIZE = 256
CHANNELS = 1
CHANNELS_3C = 3
DATADIR_IMAGES = '../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/png_images/'
DATADIR_MASKS = '../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/png_masks/'
DF_TEST = pd.read_csv('../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_test_images.csv')
DF_TRAIN = pd.read_csv('../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_train_images.csv')

# 4. Separating test and training groups

I. Let's consult and assign the size of the groups

In [ ]:
LEN_TRAIN_NO_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 0].shape[0]
LEN_TRAIN_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 1].shape[0]
LEN_TEST_NO_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 0].shape[0]
LEN_TEST_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 1].shape[0]

II. Defining empty lists for segmentation and classification tasks

In [ ]:
IMAGE_TRAIN_PNEUMO = list()
IMAGE_TRAIN_NO_PNEUMO = list()
IMAGE_TEST_PNEUMO = list()
IMAGE_TEST_NO_PNEUMO = list()
MASKS_TRAIN_PNEUMO = list()
MASKS_TRAIN_NO_PNEUMO = list()
MASKS_TEST_PNEUMO = list()
MASKS_TEST_NO_PNEUMO = list()
CLASS_IMAGE_TRAIN_PNEUMO = list()
CLASS_IMAGE_TRAIN_NO_PNEUMO = list()
CLASS_IMAGE_TEST_PNEUMO = list()
CLASS_IMAGE_TEST_NO_PNEUMO = list()

# 4. Loading data

I. Train group with pneumothorax

In [ ]:
for i in tqdm(range(0,int(LEN_TRAIN_PNEUMO))):
    DF_TRAIN_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 1]
    IMAGE_NAME = DF_TRAIN_PNEUMO.iloc[i,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    PATH_MASKS = DATADIR_MASKS + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    masks = cv2.imread(os.path.join(PATH_MASKS),cv2.IMREAD_GRAYSCALE) 
    masks = cv2.resize(masks, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    IMAGE_TRAIN_PNEUMO.append(image)
    MASKS_TRAIN_PNEUMO.append(masks)
    CLASS_IMAGE_TRAIN_PNEUMO.append(1)

II. Train group without pneumothorax

In [ ]:
for j in tqdm(range(0,int(LEN_TRAIN_PNEUMO))):
    DF_TRAIN_NO_PNEUMO = DF_TRAIN[DF_TRAIN['has_pneumo'] == 0]
    IMAGE_NAME = DF_TRAIN_NO_PNEUMO.iloc[j,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    PATH_MASKS = DATADIR_MASKS + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    masks = cv2.imread(os.path.join(PATH_MASKS),cv2.IMREAD_GRAYSCALE)
    masks = cv2.resize(masks, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    IMAGE_TRAIN_NO_PNEUMO.append(image)
    MASKS_TRAIN_NO_PNEUMO.append(masks)
    CLASS_IMAGE_TRAIN_NO_PNEUMO.append(0)

III. Test group with pneumothorax

In [ ]:
for k in tqdm(range(0,int(LEN_TEST_PNEUMO))):
    DF_TEST_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 1]
    IMAGE_NAME = DF_TEST_PNEUMO.iloc[k,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    PATH_MASKS = DATADIR_MASKS + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    masks = cv2.imread(os.path.join(PATH_MASKS),cv2.IMREAD_GRAYSCALE) 
    masks = cv2.resize(masks, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    IMAGE_TEST_PNEUMO.append(image)
    MASKS_TEST_PNEUMO.append(masks)
    CLASS_IMAGE_TEST_PNEUMO.append(1)

IV. Test group without pneumothorax

In [ ]:
for l in tqdm(range(0,int(LEN_TEST_PNEUMO))):
    DF_TEST_NO_PNEUMO = DF_TEST[DF_TEST['has_pneumo'] == 0]
    IMAGE_NAME = DF_TEST_NO_PNEUMO.iloc[l,0]
    PATH_IMAGES = DATADIR_IMAGES + IMAGE_NAME
    PATH_MASKS = DATADIR_MASKS + IMAGE_NAME
    image = cv2.imread(os.path.join(PATH_IMAGES),cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    masks = cv2.imread(os.path.join(PATH_MASKS),cv2.IMREAD_GRAYSCALE)
    masks = cv2.resize(masks, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    IMAGE_TEST_NO_PNEUMO.append(image)
    MASKS_TEST_NO_PNEUMO.append(masks)
    CLASS_IMAGE_TEST_NO_PNEUMO.append(0)

# 5. Converting to numpy format, reshaping and normalizing to the range 0-1

In [ ]:
IMAGE_TRAIN_PNEUMO = np.array(IMAGE_TRAIN_PNEUMO)
IMAGE_TRAIN_PNEUMO = IMAGE_TRAIN_PNEUMO.reshape((len(IMAGE_TRAIN_PNEUMO), SIZE, SIZE, 1))
IMAGE_TRAIN_PNEUMO = IMAGE_TRAIN_PNEUMO.astype('float16') / 255

IMAGE_TRAIN_NO_PNEUMO = np.array(IMAGE_TRAIN_NO_PNEUMO)
IMAGE_TRAIN_NO_PNEUMO = IMAGE_TRAIN_NO_PNEUMO.reshape((len(IMAGE_TRAIN_NO_PNEUMO), SIZE, SIZE, 1))
IMAGE_TRAIN_NO_PNEUMO = IMAGE_TRAIN_NO_PNEUMO.astype('float16') / 255

IMAGE_TEST_PNEUMO = np.array(IMAGE_TEST_PNEUMO)
IMAGE_TEST_PNEUMO = IMAGE_TEST_PNEUMO.reshape((len(IMAGE_TEST_PNEUMO), SIZE, SIZE, 1))
IMAGE_TEST_PNEUMO = IMAGE_TEST_PNEUMO.astype('float16') / 255

IMAGE_TEST_NO_PNEUMO = np.array(IMAGE_TEST_NO_PNEUMO)
IMAGE_TEST_NO_PNEUMO = IMAGE_TEST_NO_PNEUMO.reshape((len(IMAGE_TEST_NO_PNEUMO), SIZE, SIZE, 1))
IMAGE_TEST_NO_PNEUMO = IMAGE_TEST_NO_PNEUMO.astype('float16') / 255

MASKS_TRAIN_PNEUMO = np.array(MASKS_TRAIN_PNEUMO)
MASKS_TRAIN_PNEUMO = MASKS_TRAIN_PNEUMO.reshape((len(MASKS_TRAIN_PNEUMO), SIZE, SIZE, 1))
MASKS_TRAIN_PNEUMO = MASKS_TRAIN_PNEUMO.astype('float16') / 255

MASKS_TRAIN_NO_PNEUMO = np.array(MASKS_TRAIN_NO_PNEUMO)
MASKS_TRAIN_NO_PNEUMO = MASKS_TRAIN_NO_PNEUMO.reshape((len(MASKS_TRAIN_NO_PNEUMO), SIZE, SIZE, 1))
MASKS_TRAIN_NO_PNEUMO = MASKS_TRAIN_NO_PNEUMO.astype('float16') / 255

MASKS_TEST_PNEUMO = np.array(MASKS_TEST_PNEUMO)
MASKS_TEST_PNEUMO = MASKS_TEST_PNEUMO.reshape((len(MASKS_TEST_PNEUMO), SIZE, SIZE, 1))
MASKS_TEST_PNEUMO = MASKS_TEST_PNEUMO.astype('float16') / 255

MASKS_TEST_NO_PNEUMO = np.array(MASKS_TEST_NO_PNEUMO)
MASKS_TEST_NO_PNEUMO = MASKS_TEST_NO_PNEUMO.reshape((len(MASKS_TEST_NO_PNEUMO), SIZE, SIZE, 1))
MASKS_TEST_NO_PNEUMO = MASKS_TEST_NO_PNEUMO.astype('float16') / 255

CLASS_IMAGE_TRAIN_PNEUMO = np.array(CLASS_IMAGE_TRAIN_PNEUMO)
CLASS_IMAGE_TRAIN_NO_PNEUMO = np.array(CLASS_IMAGE_TRAIN_NO_PNEUMO)
CLASS_IMAGE_TEST_PNEUMO = np.array(CLASS_IMAGE_TEST_PNEUMO)
CLASS_IMAGE_TEST_NO_PNEUMO = np.array(CLASS_IMAGE_TEST_NO_PNEUMO)

# 6. Concatenated separation of training and test groups

In [ ]:
X_train = np.concatenate((IMAGE_TRAIN_PNEUMO,IMAGE_TRAIN_NO_PNEUMO))
y_train = np.concatenate((MASKS_TRAIN_PNEUMO,MASKS_TRAIN_NO_PNEUMO))
X_test = np.concatenate((IMAGE_TEST_PNEUMO,IMAGE_TEST_NO_PNEUMO))
y_test = np.concatenate((MASKS_TEST_PNEUMO,MASKS_TEST_NO_PNEUMO))
class_train = np.concatenate((CLASS_IMAGE_TRAIN_PNEUMO,CLASS_IMAGE_TRAIN_NO_PNEUMO))
class_test = np.concatenate((CLASS_IMAGE_TEST_PNEUMO,CLASS_IMAGE_TEST_NO_PNEUMO))

In [ ]:
y_test.shape

In [ ]:
number = 0
predicted_rx = np.squeeze(y_train[number].astype(np.float32))
plt.imshow(predicted_rx, cmap='gray')
plt.show()



In [ ]:
number = 3
fig, ax = plt.subplots(1,2, figsize = (16,8))
ax[0].imshow(X_test[number], cmap='gray')
ax[1].imshow(y_test[number], cmap='gray')
plt.show()

In [ ]:
import gc
del IMAGE_TRAIN_PNEUMO 
del IMAGE_TRAIN_NO_PNEUMO
del MASKS_TRAIN_PNEUMO 
del MASKS_TRAIN_NO_PNEUMO
del IMAGE_TEST_PNEUMO
del IMAGE_TEST_NO_PNEUMO
del MASKS_TEST_PNEUMO
del MASKS_TEST_NO_PNEUMO
gc.collect()

**Observation:** *We will make some changes in the groups in order to adapt to the VGG19*

In [ ]:
X_train_3C = list()
X_test_3C = list()

for img in X_train:
    img_3C = np.concatenate((img,)*3, axis=-1)
    X_train_3C.append(img_3C)
    
for img in X_test:
    img_3C = np.concatenate((img,)*3, axis=-1)
    X_test_3C.append(img_3C)
    
X_train_3C = np.array(X_train_3C)
X_test_3C = np.array(X_test_3C)

# 7. Transforming targets into categorical classes

In [ ]:
binarizer = LabelBinarizer()

labels_train = binarizer.fit_transform(class_train)
labels_train = to_categorical(labels_train)
labels_test = binarizer.fit_transform(class_test)
labels_test = to_categorical(labels_test)

# 8. Defining the metrics and losses for the structure segmentation model 

In [ ]:
def iou_score(y_pred, y_true, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true, -1) + K.sum(y_pred, -1) - intersection
    iou = (intersection + smooth)/(union + smooth)
    return iou

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred) 

# 9. Structuring and training the classification network using VGG19

In [ ]:
vgg_19 = VGG19(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, CHANNELS_3C))
vgg_19.summary()

In [ ]:
for i, layer in enumerate(vgg_19.layers):
    print(i,layer)

Don't compile if you'll training with RF

In [ ]:
for layer in vgg_19.layers[:17]:
    layer.trainable = False

In [ ]:
vgg_19.summary()

In [ ]:
classifier = models.Sequential()
classifier.add(vgg_19)
classifier.add(layers.GlobalAveragePooling2D())
classifier.add(layers.BatchNormalization())
classifier.add(layers.Flatten())
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Dropout(0.6))
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Dropout(0.4))
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Dropout(0.3))
classifier.add(layers.Dense(2, activation='softmax'))
classifier.summary()
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1)
datagen.fit(X_train_3C)
data_augmentation = datagen.flow(X_train_3C, labels_train, batch_size=16)

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_delta=1e-5, patience=5, verbose=1), ModelCheckpoint('Classifier_pneumo.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
result_class = classifier.fit(data_augmentation, validation_data=(X_test_3C, labels_test), callbacks=callbacks, epochs=100)

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_delta=1e-5, patience=5, verbose=1), ModelCheckpoint('Classifier_pneumo.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
result_class = classifier.fit(X_train_3C, labels_train, validation_data=(X_test_3C, labels_test),batch_size=16, epochs=100, callbacks = callbacks)

In [ ]:
classifier.save('Classifier_complete.h5')

# 9.1 - VGG19 to RF/SVM

In [ ]:
feature_extractor=vgg_19.predict(X_train_3C)

In [ ]:
feature_extractor.shape

In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
features.shape

SVM

In [ ]:
from sklearn.svm import SVC
SVM_model = SVC(kernel = 'rbf', random_state = 1, C = 2.0, probability=True)
SVM_model.fit(features, class_train)

In [ ]:
X_test_feature = vgg_19.predict(X_test_3C)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

prediction_SVM = SVM_model.predict(X_test_features)

RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 0)
RF_model.fit(features, labels_train)

In [ ]:
X_test_feature = vgg_19.predict(X_test_3C)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

prediction_RF = RF_model.predict(X_test_features)

# 10. Loading and evaluating the classification model

To normal vgg19

In [ ]:
load_classifier = load_model('../input/package/Classifier_pneumo (1).h5')
load_classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
load_classifier.evaluate(X_test_3C, labels_test,batch_size=16, verbose=1)

In [ ]:
class_pred = load_classifier.predict(X_test_3C, verbose=1)

In [ ]:
class_pred = class_pred.round()
class_true = labels_test
print(classification_report(class_pred, class_true))

To vgg19 + SVM

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(class_test, prediction_SVM.round()))

In [ ]:
class_pred = prediction_SVM.round()
class_true = class_test
print(classification_report(class_pred, class_true))

To vgg19 + RF

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(labels_test, prediction_RF.round()))

In [ ]:
class_pred = prediction_RF.round()
class_true = labels_test
print(classification_report(class_pred, class_true))

# 13. Doing some predictions

In [ ]:
class_true[1]

In [ ]:
prediction = classifier.predict(X_test_3C[[1]].astype(np.float64), verbose=1)
prediction.round()

# 12. Structuring and training the Xnet network for segmentation

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strategy.scope():    
    input_shape=(SIZE,SIZE,1)
    classes=1
    kernel_size = 3
    filter_depth = (64,128,256,512,0)

    img_input = Input(shape=input_shape)

    conv1 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(img_input)
    batch1 = BatchNormalization()(conv1)
    act1 = Activation("relu")(batch1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(act1)

    conv2 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(pool1)
    batch2 = BatchNormalization()(conv2)
    act2 = Activation("relu")(batch2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(act2)

    conv3 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool2)
    batch3 = BatchNormalization()(conv3)
    act3 = Activation("relu")(batch3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(act3)

    conv4 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool3)
    batch4 = BatchNormalization()(conv4)
    act4 = Activation("relu")(batch4)

    conv5 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act4)
    batch5 = BatchNormalization()(conv5)
    act5 = Activation("relu")(batch5)

    up6 = UpSampling2D(size=(2, 2))(act5)
    conv6 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up6)
    batch6 = BatchNormalization()(conv6)
    act6 = Activation("relu")(batch6)
    concat6 = concatenate([act3,act6])

    up7 = UpSampling2D(size=(2, 2))(concat6)
    conv7 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up7)
    batch7 = BatchNormalization()(conv7)
    act7 = Activation("relu")(batch7)
    concat7 = concatenate([act2,act7])

    conv8 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(concat7)
    batch8 = BatchNormalization()(conv8)
    act8 = Activation("relu")(batch8)
    pool8 = MaxPooling2D(pool_size=(2, 2))(act8)

    conv9 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool8)
    batch9 = BatchNormalization()(conv9)
    act9 = Activation("relu")(batch9)
    pool9 = MaxPooling2D(pool_size=(2, 2))(act9)

    conv10 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool9)
    batch10 = BatchNormalization()(conv10)
    act10 = Activation("relu")(batch10)

    conv11 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act10)
    batch11 = BatchNormalization()(conv11)
    act11 = Activation("relu")(batch11)

    up12 = UpSampling2D(size=(2, 2))(act11)
    conv12 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up12)
    batch12 = BatchNormalization()(conv12)
    act12 = Activation("relu")(batch12)
    concat12 = concatenate([act9,act12])

    up13 = UpSampling2D(size=(2, 2))(concat12)
    conv13 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up13)
    batch13 = BatchNormalization()(conv13)
    act13 =  Activation("relu")(batch13)
    concat13 = concatenate([act8,act13])

    up14 = UpSampling2D(size=(2, 2))(concat13)
    conv14 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(up14)
    batch14 = BatchNormalization()(conv14)
    act14 = Activation("relu")(batch14)
    concat14 = concatenate([act1,act14])

    output_xnet = Conv2D(1, (1, 1), activation='sigmoid')(concat14)

    model = Model(img_input, output_xnet)
    model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=[dice_coef_loss])
    model.summary()

In [ ]:
callbacks = [ModelCheckpoint('Segmenter_pneumo.h5', monitor='val_dice_coef_loss', verbose=1, save_best_only=True, mode='min')]
results = model.fit(IMAGE_TRAIN_PNEUMO, MASKS_TRAIN_PNEUMO, validation_split=0.25,batch_size=8 * tpu_strategy.num_replicas_in_sync, epochs=300, callbacks = callbacks)

In [ ]:
plt.plot(results.history['dice_coef_loss'], label='train')
plt.plot(results.history['val_dice_coef_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
model.save('Segmenter_complete.h5')

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model_autoencoder = load_model('./Segmenter_pneumo_yes.h5', custom_objects=dependencies)
loaded_model_autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

In [ ]:
number = 16
im = model.predict(IMAGE_TEST_PNEUMO[[number]].astype(np.float32))
predicted_rx = np.squeeze(im.astype(np.float32))
plt.imshow(predicted_rx, cmap='gray')
plt.show()

In [ ]:
number = 400
im = loaded_model_autoencoder.predict(X_test[[number]].astype(np.float16))
im = np.squeeze(im.astype(np.float16))
plt.imsave('predicted_000.jpg', predicted_rx, cmap='gray')
img = cv2.imread('./predicted_000.jpg',1)
img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
fig, ax = plt.subplots(1,1, figsize = (8,4))
ax.imshow(thresh, cmap='gray')
plt.show()

# 14. Loading and evaluating the segmentation model

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss, 'iou_score': iou_score}
loaded_model = load_model('../input/my-segmenter/my_segmenter (1).h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

In [ ]:
results = loaded_model.evaluate(X_train, y_train,batch_size=16, verbose=1)
print("Loss and dice_coef_loss:", results)

# 15. Doing some predictions

In [ ]:
number = 124

In [ ]:
plt.imshow(IMAGE_TEST_PNEUMO[number], cmap = 'gray')
plt.show()

In [ ]:
plt.imshow(MASKS_TEST_PNEUMO[number], cmap = 'gray')
plt.show()

Heat map

In [ ]:
image_rx = np.squeeze(IMAGE_TEST_PNEUMO[number].astype(np.float64))
plt.imsave('img_000.jpg', image_rx, cmap='gray')
img_000 = cv2.imread('./img_000.jpg',1)


segmentation = loaded_model.predict(X_test[[number]].astype(np.float64),verbose = 1)
segmentation = np.squeeze(segmentation.astype(np.float64))
plt.imsave('mask_000.jpg', segmentation, cmap='gray')
img = cv2.imread('./mask_000.jpg',1)
#img_grey = img[:,:,0]
#min_ = img_grey.min()
#max_ = img_grey.max()
#ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#plt.imsave('mask_thresh_000.jpg', thresh, cmap='gray')
#img = cv2.imread('./mask_thresh_000.jpg',1)

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
heatmap_img = cv2.applyColorMap(gray_img, cv2.COLORMAP_JET)
added = cv2.addWeighted(heatmap_img, 0.3, img_000, 0.7, 0)
plt.imshow(added)
plt.show()

Mask

In [ ]:
img = cv2.imread('./mask_000.jpg')
img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
plt.imshow(np.squeeze((thresh).astype(np.uint8)),cmap = 'gray')
plt.show()

# 16. Interesting things to publish

In [ ]:
fig, axs = plt.subplots(5,3, figsize=(30,70))
n_image = [8, 87, 250, 198, 45]
paths = ['./mask_8.jpg', './mask_87.jpg','./mask_250.jpg', './mask_198.jpg', './mask_45.jpg']

for i in range(0,5):
    axs[i][0].imshow(IMAGE_TEST_PNEUMO[n_image[i]], cmap = 'gray')
    axs[i][0].title.set_text('Chest x-ray')
    axs[i][1].imshow(MASKS_TEST_PNEUMO[n_image[i]], cmap = 'gray')
    axs[i][1].title.set_text('Original Mask')
    img = cv2.imread(paths[i])
    img_grey = img[:,:,0]
    min_ = img_grey.min()
    max_ = img_grey.max()
    ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    axs[i][2].imshow(np.squeeze((thresh).astype(np.uint8)),cmap = 'gray')
    axs[i][2].title.set_text('AI predicted mask')
    
fig.savefig('images.png')

In [ ]:
img1 = cv2.imread('./mask_45.jpg')
for row in range(SIZE):
    for col in range(SIZE):
        if np.all(img1[row,col] >= [50,50,50]):
            img1[row,col] = [255,0,0]
            
img2 = cv2.imread('./img_45.jpg')

sum_ = cv2.addWeighted( img1, 0.3, img2, 1, 0)
fig1, axs = plt.subplots(1,1, figsize=(4,4))          
plt.imshow(sum_)
plt.show()
fig1.savefig('Image_red.png') 

# 17. Lung segmentation from Chest X-Ray 

In [ ]:
import numpy as np 
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import os
from cv2 import imread
import cv2
import matplotlib.pyplot as plt

image_path = os.path.join("../input/chest-xray-masks-and-labels/Lung Segmentation/CXR_png")
mask_path = os.path.join("../input/chest-xray-masks-and-labels/Lung Segmentation/","masks/")

images = os.listdir(image_path)
mask = os.listdir(mask_path)

Way adapted to import the dataset: https://www.kaggle.com/nikhilpandey360/lung-segmentation-from-chest-x-ray-dataset

In [ ]:
mask = [fName.split(".png")[0] for fName in mask]
image_file_name = [fName.split("_mask")[0] for fName in mask]
check = [i for i in mask if "mask" in i]

testing_files = set(os.listdir(image_path)) & set(os.listdir(mask_path))
training_files = check

#len(image_file_name) - len(training_files)
#len(testing_files)
#image_file_name
#training_files
#testing_files

In [ ]:
im_train = list()
mask_train = list()
im_test = list()
mask_test = list()

for i in tqdm(testing_files): 
    im = cv2.imread(os.path.join(image_path,i),cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    im = cv2.equalizeHist(im)
    mask = cv2.imread(os.path.join(mask_path,i),cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    im_test.append(im)
    mask_test.append(mask)
    
for i in tqdm(training_files): 
    im = cv2.imread(os.path.join(image_path,i.split("_mask")[0]+".png"),cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (SIZE,SIZE), interpolation = cv2.INTER_AREA)
    im = cv2.equalizeHist(im)
    mask = cv2.imread(os.path.join(mask_path,i+".png"),cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (SIZE,SIZE), interpolation = cv2.INTER_AREA)    
    im_train.append(im)
    mask_train.append(mask)

In [ ]:
SIZE = 128

im_train = np.array(im_train)
mask_train = np.array(mask_train)
im_test = np.array(im_test)
mask_test = np.array(mask_test)

im_train = im_train.reshape((len(im_train), SIZE, SIZE, 1))
mask_train = mask_train.reshape((len(mask_train), SIZE, SIZE, 1))
im_test = im_test.reshape((len(im_test), SIZE, SIZE, 1))
mask_test = mask_test.reshape((len(mask_test), SIZE, SIZE, 1))

X_train = im_train.astype('float32') / 255
y_train = mask_train.astype('float32') / 255
X_test = im_test.astype('float32') / 255
y_test = mask_test.astype('float32') / 255

In [ ]:
input_shape=(SIZE,SIZE,1)
classes=1
kernel_size = 3
filter_depth = (64,128,256,512,0)
    
img_input = Input(shape=input_shape)

conv1 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(img_input)
batch1 = BatchNormalization()(conv1)
act1 = Activation("relu")(batch1)
pool1 = MaxPooling2D(pool_size=(2, 2))(act1)

conv2 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(pool1)
batch2 = BatchNormalization()(conv2)
act2 = Activation("relu")(batch2)
pool2 = MaxPooling2D(pool_size=(2, 2))(act2)

conv3 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool2)
batch3 = BatchNormalization()(conv3)
act3 = Activation("relu")(batch3)
pool3 = MaxPooling2D(pool_size=(2, 2))(act3)

conv4 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool3)
batch4 = BatchNormalization()(conv4)
act4 = Activation("relu")(batch4)

conv5 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act4)
batch5 = BatchNormalization()(conv5)
act5 = Activation("relu")(batch5)

up6 = UpSampling2D(size=(2, 2))(act5)
conv6 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up6)
batch6 = BatchNormalization()(conv6)
act6 = Activation("relu")(batch6)
concat6 = concatenate([act3,act6])

up7 = UpSampling2D(size=(2, 2))(concat6)
conv7 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up7)
batch7 = BatchNormalization()(conv7)
act7 = Activation("relu")(batch7)
concat7 = concatenate([act2,act7])

conv8 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(concat7)
batch8 = BatchNormalization()(conv8)
act8 = Activation("relu")(batch8)
pool8 = MaxPooling2D(pool_size=(2, 2))(act8)

conv9 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(pool8)
batch9 = BatchNormalization()(conv9)
act9 = Activation("relu")(batch9)
pool9 = MaxPooling2D(pool_size=(2, 2))(act9)

conv10 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(pool9)
batch10 = BatchNormalization()(conv10)
act10 = Activation("relu")(batch10)

conv11 = Conv2D(filter_depth[3], (kernel_size, kernel_size), padding="same")(act10)
batch11 = BatchNormalization()(conv11)
act11 = Activation("relu")(batch11)

up12 = UpSampling2D(size=(2, 2))(act11)
conv12 = Conv2D(filter_depth[2], (kernel_size, kernel_size), padding="same")(up12)
batch12 = BatchNormalization()(conv12)
act12 = Activation("relu")(batch12)
concat12 = concatenate([act9,act12])

up13 = UpSampling2D(size=(2, 2))(concat12)
conv13 = Conv2D(filter_depth[1], (kernel_size, kernel_size), padding="same")(up13)
batch13 = BatchNormalization()(conv13)
act13 =  Activation("relu")(batch13)
concat13 = concatenate([act8,act13])

up14 = UpSampling2D(size=(2, 2))(concat13)
conv14 = Conv2D(filter_depth[0], (kernel_size, kernel_size), padding="same")(up14)
batch14 = BatchNormalization()(conv14)
act14 = Activation("relu")(batch14)
concat14 = concatenate([act1,act14])

output_xnet = Conv2D(1, (1, 1), activation='sigmoid')(concat14)

model = Model(img_input, output_xnet)
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=[dice_coef_loss])
model.summary()

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='val_dice_coef_loss', factor=0.1, min_delta=1e-5, patience=8, verbose=1), ModelCheckpoint('Segmenter_lung.h5', monitor='val_dice_coef_loss', verbose=1, save_best_only=True, mode='min')]
results = model.fit(X_train, y_train, validation_split=0.1,batch_size=16, epochs=300, callbacks = callbacks)

In [ ]:
plt.plot(results.history['dice_coef_loss'], label='train')
plt.plot(results.history['val_dice_coef_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
model.save('Segmenter_lung_complete.h5')

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model = load_model('../input/segmenter-mais-eficente-300/Segmenter_lung (2).h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

In [ ]:
results = loaded_model.evaluate(X_test, y_test,batch_size=16, verbose=1)
print("Loss and dice_coef_loss:", results)

In [ ]:
number = 80
image_rx = np.squeeze(X_test[number].astype(np.float64))
plt.imsave('img_000.jpg', image_rx, cmap='gray')
img_000 = cv2.imread('./img_000.jpg',1)


segmentation = model.predict(X_test[[number]].astype(np.float64),verbose = 1)
segmentation = np.squeeze(segmentation.astype(np.float64))
plt.imsave('mask_000.jpg', segmentation, cmap='gray')
img = cv2.imread('./mask_000.jpg',1)
img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
plt.imsave('mask_thresh_000.jpg', thresh, cmap='gray')
img = cv2.imread('./mask_thresh_000.jpg',1)

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
heatmap_img = cv2.applyColorMap(gray_img, cv2.COLORMAP_JET)
added = cv2.addWeighted(heatmap_img, 0.3, img_000, 0.7, 0)
plt.imshow(added)
plt.show()

In [ ]:
img_000 = cv2.imread('./img_000.jpg',1)
plt.imshow(img_000)
plt.show()

In [ ]:
mask_rx = np.squeeze(y_test[number].astype(np.float64))
plt.imsave('mask_000.jpg', mask_rx, cmap='gray')
mask_000 = cv2.imread('./mask_000.jpg',1)
plt.imshow(mask_000)
plt.show()

In [ ]:
mask_t_000 = cv2.imread('mask_thresh_000.jpg',1)
plt.imshow(mask_t_000)
plt.show()

# 18. Lung segmentation from pneumothorax chest X-Ray 

In [ ]:
X_test = np.concatenate((IMAGE_TEST_PNEUMO,IMAGE_TEST_NO_PNEUMO))

In [ ]:
X_test = im_test.astype('float32') / 255

In [ ]:
X_test_R = list()

for img in X_test:
    img_R = np.concatenate((img,)*3, axis=-1)
    X_test_R.append(img_R)
    
X_test_R = np.array(X_test_R)

In [ ]:
len(X_test_R)

In [ ]:
number = 1
image_rx = np.squeeze(X_test_[number].astype(np.float64))
plt.imsave('img_000.jpg', image_rx, cmap='gray')
img_000 = cv2.imread('./img_000.jpg',1)

segmentation = model.predict(X_test_[[number]].astype(np.float64),verbose = 1)
segmentation = np.squeeze(segmentation.astype(np.float64))
plt.imsave('mask_000.jpg', segmentation, cmap='gray')
img = cv2.imread('./mask_000.jpg',1)
img_grey = img[:,:,0]
min_ = img_grey.min()
max_ = img_grey.max()
ret1, thresh = cv2.threshold(img_grey, min_, max_, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
plt.imsave('mask_thresh_000.jpg', thresh, cmap='gray')
img = cv2.imread('./mask_thresh_000.jpg',1)

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
heatmap_img = cv2.applyColorMap(gray_img, cv2.COLORMAP_JET)
added = cv2.addWeighted(heatmap_img, 0.3, img_000, 0.7, 0)
plt.imshow(added)
plt.show()

# 19. Importing relevant packages for radiomics

In [ ]:
pip install pyradiomics

In [ ]:
from __future__ import print_function
import SimpleITK as sitk
import six
from radiomics import featureextractor, getTestCase, glcm, glrlm, glszm, imageoperations, shape, getImageTypes, getFeatureClasses, getParameterValidationFiles

# 20. Structuring the radiomics analysis

In [ ]:
X_test_ = list()

for img in X_train:
    img_R = np.concatenate((img,)*3, axis=-1)
    X_test_.append(img_R)
    
X_test_ = np.array(X_test_)

In [ ]:
len_images = len(X_test_)
len_images

In [ ]:
len(X_train_R)

In [ ]:
os.makedirs('./images')

In [ ]:
os.makedirs('./input/images')

In [ ]:
for i in range(0,len_images):    
    image_chest = np.squeeze(X_train_R[i].astype(np.float64))
    plt.imsave('./images/Image_chest'+str(i)+'.jpg', image_chest, cmap = 'gray')
    image_segmentation = model.predict(X_test_[[i]].astype(np.float64),verbose = 1)
    image_segmentation = np.squeeze(image_segmentation.astype(np.float64))
    plt.imsave('./images/Mask_chest'+str(i)+'.jpg', image_segmentation, cmap = 'gray')

In [ ]:
img1 = cv2.imread('.images/Image_chest.jpg',0)
img2 = cv2.imread('.images/Mask_chest.jpg',0)

img = ["",img1,img2]
fig=plt.figure(figsize=(8, 8))
columns = 2
rows = 1
for i in range(1, columns*rows+1):
    img_ = img[i]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img_,cmap = 'gray')
plt.show()

In [ ]:
#for i in range(0,len_images): 
#    image = sitk.ReadImage('./Image_chest'+str(i)+'.jpg', sitk.sitkInt8)
#    mask = sitk.ReadImage('./Mask_chest'+str(i)+'.jpg', sitk.sitkInt8)

In [ ]:
applyLog = False
applyWavelet = False

In [ ]:
settings = {'binWidth':25,'label': 1, 'interpolator': sitk.sitkBSpline, 'resampledPixelSpacing':None}

In [ ]:
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

In [ ]:
len(featureVector)

In [ ]:
featureVector = []
for i in range(0,len_images): 
    image = sitk.ReadImage('./images/Image_chest'+str(i)+'.jpg', sitk.sitkInt8)
    mask = sitk.ReadImage('./images/Mask_chest'+str(i)+'.jpg', sitk.sitkInt8)
    result = pd.Series(extractor.execute(image, mask))
    featureVector.append(result)
    print(i)

In [ ]:
len(featureVector)

In [ ]:
df = pd.DataFrame(featureVector)
df

In [ ]:
df.to_csv('Features', sep=',', encoding='utf-8')

In [ ]:
df = pd.read_csv('./Features')

In [ ]:
df

In [ ]:
print(df.columns.tolist())

In [ ]:
df = df.drop(columns = ['Unnamed: 0', 'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'])
df

In [ ]:
pd.set_option('display.max_columns', df.shape[0]+1)
df

# 20.1 - Feature reduction

In [ ]:
pip install shap

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
import shap

In [ ]:
df = pd.read_csv('../input/features-radiomics/Features_10k.csv')

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df, class_train, test_size=0.25, random_state=12)

In [ ]:
y_C_train.dtype

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_F_train, y_C_train)
#explainer = shap.TreeExplainer(rf)
#shap_values = explainer.shap_values(X_F_train)
#shap.summary_plot(shap_values, X_F_test, plot_type="bar")

In [ ]:
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_C_test, prediction.round()))

In [ ]:
sorted_idx = rf.feature_importances_.argsort()
fig = plt.subplots(figsize=(10,26))
plt.barh(df.columns.to_list(), rf.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

In [ ]:
import seaborn

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10,10))
seaborn.clustermap(df.corr(), metric="correlation", cmap = 'coolwarm')

In [ ]:
plt.figure(figsize=(10,10))
seaborn.heatmap(df.corr(),annot = False, cmap = 'coolwarm')

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

rf_ = RandomForestClassifier() 
rfecv = RFECV(estimator=rf_, step=1, cv=5,scoring='accuracy')
rfecv = rfecv.fit(X_F_train, y_C_train)

In [ ]:
print('Optimal number of features :', rfecv.n_features_)
print('Best features :', X_F_train.columns[rfecv.support_])

In [ ]:
df2=df.loc[:,['original_firstorder_10Percentile', 'original_firstorder_90Percentile',
       'original_firstorder_Energy', 'original_firstorder_Entropy',
       'original_firstorder_InterquartileRange',
       'original_firstorder_Kurtosis', 'original_firstorder_Mean',
       'original_firstorder_Median',
       'original_firstorder_RobustMeanAbsoluteDeviation',
       'original_firstorder_RootMeanSquared',
       'original_firstorder_TotalEnergy', 'original_firstorder_Uniformity',
       'original_firstorder_Variance', 'original_glcm_Autocorrelation',
       'original_glcm_ClusterProminence', 'original_glcm_ClusterShade',
       'original_glcm_ClusterTendency', 'original_glcm_Contrast',
       'original_glcm_Correlation', 'original_glcm_DifferenceAverage',
       'original_glcm_DifferenceEntropy', 'original_glcm_DifferenceVariance',
       'original_glcm_Id', 'original_glcm_Idm', 'original_glcm_Idmn',
       'original_glcm_Idn', 'original_glcm_Imc1', 'original_glcm_Imc2',
       'original_glcm_InverseVariance', 'original_glcm_JointAverage',
       'original_glcm_JointEnergy', 'original_glcm_JointEntropy',
       'original_glcm_MaximumProbability', 'original_glcm_SumAverage',
       'original_glcm_SumEntropy', 'original_glcm_SumSquares',
       'original_gldm_DependenceEntropy',
       'original_gldm_DependenceNonUniformity',
       'original_gldm_DependenceVariance',
       'original_gldm_GrayLevelNonUniformity',
       'original_gldm_LargeDependenceEmphasis',
       'original_gldm_LargeDependenceHighGrayLevelEmphasis',
       'original_gldm_LargeDependenceLowGrayLevelEmphasis',
       'original_gldm_LowGrayLevelEmphasis',
       'original_gldm_SmallDependenceEmphasis',
       'original_gldm_SmallDependenceHighGrayLevelEmphasis',
       'original_gldm_SmallDependenceLowGrayLevelEmphasis',
       'original_glrlm_GrayLevelNonUniformity',
       'original_glrlm_GrayLevelNonUniformityNormalized',
       'original_glrlm_GrayLevelVariance', 'original_glrlm_LongRunEmphasis',
       'original_glrlm_LongRunLowGrayLevelEmphasis',
       'original_glrlm_LowGrayLevelRunEmphasis', 'original_glrlm_RunEntropy',
       'original_glrlm_RunLengthNonUniformity',
       'original_glrlm_RunLengthNonUniformityNormalized',
       'original_glrlm_RunVariance', 'original_glrlm_ShortRunEmphasis',
       'original_glrlm_ShortRunHighGrayLevelEmphasis',
       'original_glrlm_ShortRunLowGrayLevelEmphasis',
       'original_glszm_GrayLevelNonUniformity',
       'original_glszm_GrayLevelNonUniformityNormalized',
       'original_glszm_GrayLevelVariance',
       'original_glszm_HighGrayLevelZoneEmphasis',
       'original_glszm_LargeAreaHighGrayLevelEmphasis',
       'original_glszm_LargeAreaLowGrayLevelEmphasis',
       'original_glszm_LowGrayLevelZoneEmphasis',
       'original_glszm_SizeZoneNonUniformity',
       'original_glszm_SizeZoneNonUniformityNormalized',
       'original_glszm_SmallAreaEmphasis',
       'original_glszm_SmallAreaHighGrayLevelEmphasis',
       'original_glszm_SmallAreaLowGrayLevelEmphasis',
       'original_glszm_ZoneEntropy', 'original_glszm_ZoneVariance',
       'original_ngtdm_Busyness', 'original_ngtdm_Coarseness',
       'original_ngtdm_Complexity', 'original_ngtdm_Contrast',
       'original_ngtdm_Strength']]

In [ ]:
df2.to_csv('Features_reduced', sep=',', encoding='utf-8')

In [ ]:
df_reduced = pd.read_csv('./Features_reduced')
df_reduced = df_reduced.drop(columns = 'Unnamed: 0', axis=1)
df_reduced

In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df_reduced, class_train, test_size=0.25, random_state=12)
rf = RandomForestClassifier()
rf.fit(X_F_train, y_C_train)
print ("Accuracy = ", metrics.accuracy_score(y_C_test, rf.predict(X_F_test)))

In [ ]:
plt.figure(figsize=(10,10))
seaborn.clustermap(df_reduced.corr(), metric="correlation", cmap = 'coolwarm')

In [ ]:
df_reduced.corr()

In [ ]:
corr_features = list()
corr_matrix = df_reduced.corr()

for i in range(len(corr_matrix .columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.3:
            colname = corr_matrix.columns[i]
            corr_features.append(str(colname))
            
df_reduced_corr = df_reduced.drop(columns = corr_features, axis=1)

In [ ]:
corr_features

In [ ]:
df_reduced_corr

In [ ]:
df_reduced_corr.to_csv('Features_reduced_corr', sep=',', encoding='utf-8')

In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df_reduced_corr, class_train, test_size=0.25, random_state=12)
rf = RandomForestClassifier()
rf.fit(X_F_train, y_C_train)
print ("Accuracy = ", metrics.accuracy_score(y_C_test, rf.predict(X_F_test)))

In [ ]:
X_F_train.shape

# 21. Assigning a BRISQUE score for each image 

In [ ]:
pip install image-quality

In [ ]:
import imquality.brisque as brisque

In [ ]:
df_features = pd.DataFrame({'Brisque_score':[]})
df_features

In [ ]:
score = []
for i in range(0,len(X_test_R)):
    score.append(brisque.score(X_test_R[i]))
df_features['Brisque_score'] = score

In [ ]:
df_features.describe()

In [ ]:
No_train = df_features

In [ ]:
No_train

In [ ]:
No_train.describe()

# 22. TEST - Segmenter using VGG, ResNet, Inception and among others as backbones

In [ ]:
pip install -U segmentation-models

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import normalize
from tensorflow.keras.metrics import MeanIoU
%env SM_FRAMEWORK=tf.keras

In [ ]:
import segmentation_models as sm
sm.set_framework('tf.keras')

In [ ]:
n_classes = 1
activation = 'sigmoid'
LR = 0.0001
optim = keras.optimizers.Adam(LR)
BATCH_SIZE = 16

dice_loss = sm.losses.DiceLoss()
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
X_train_R = list()

for img in X_train:
    img_R = np.concatenate((img,)*3, axis=-1)
    X_train_R.append(img_R)
    
X_train_R = np.array(X_train_R)

In [ ]:
X_test_R = list()

for img in X_test:
    img_R = np.concatenate((img,)*3, axis=-1)
    X_test_R.append(img_R)
    
X_test_R = np.array(X_test_R)

In [ ]:
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

x_train = preprocess_input1(X_train_R)
x_val = preprocess_input1(X_test_R)

model = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes = n_classes, activation = activation)
model.compile('Adam', loss='binary_crossentropy', metrics=[dice_coef_loss])
print(model.summary())

In [ ]:
callbacks = [ModelCheckpoint('Segmenter_lung.h5', monitor='val_dice_coef_loss', verbose=1, save_best_only=True, mode='min')]
history = model.fit(x_train, y_train, batch_size = 16, epochs = 800, verbose = 1, validation_data = (x_val,y_test), callbacks = callbacks)

In [ ]:
model.save('resnet34_segmenter.h5')

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
model = load_model('../input/package/resnet34_segmenter (1).h5', custom_objects=dependencies)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])

In [ ]:
history = None
model = None

In [ ]:
dependencies = {'dice_coef_loss': dice_coef_loss}
loaded_model = load_model('../input/segmenter-mais-eficente-300/Segmenter_lung (2).h5', custom_objects=dependencies)
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef_loss])
results = loaded_model.evaluate(x_val,y_test,batch_size=16, verbose=1)
print("Loss and dice_coef_loss:", results)

# 23. VGG16 Feature importance

In [ ]:
from tensorflow.keras.applications import VGG16
vgg_16 = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, CHANNELS_3C))
vgg_16.summary()

In [ ]:
for i, layer in enumerate(vgg_16.layers):
    print(i,layer)

In [ ]:
for layer in vgg_16.layers[:19]:
    layer.trainable = False

In [ ]:
vgg_16.summary()

In [ ]:
X_train_feature = vgg_16.predict(X_train_3C)

In [ ]:
X_train_feature.shape

In [ ]:
X_train_features = X_train_feature.reshape(X_train_feature.shape[0], -1)

In [ ]:
X_test_features.shape

In [ ]:
len(X_train_features[1])

In [ ]:
df_vgg16 = pd.DataFrame(X_train_features)


In [ ]:
df_vgg16

In [ ]:
df_vgg16.to_csv('Features_vgg16', sep=',', encoding='utf-8')

In [ ]:
df_vgg16 = pd.read_csv('../input/features-vgg16/Features_vgg16')

In [ ]:
target_train = pd.DataFrame([class_train])
target_train.to_csv('Targets_train.csv', sep=',', encoding='utf-8')

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df_vgg16, class_train, test_size=0.30)

In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df_vgg16, class_train, test_size=0.25, random_state=12)
rf = RandomForestClassifier()
rf.fit(X_F_train, y_C_train)

In [ ]:
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_C_test, prediction.round()))

In [ ]:
print("MCC: ", metrics.matthews_corrcoef(y_C_test, prediction.round()))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn

confusion_matrix = confusion_matrix(y_C_test, prediction.round())
confusion_matrix

In [ ]:
sn.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X_F_train, y_C_train)
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(df_vgg16)

In [ ]:
train_reduced = pd.DataFrame(X_new)
train_reduced.to_csv('train_reduced.csv', sep=',', encoding='utf-8')

In [ ]:
train_reduced = pd.read_csv('../input/festures-vgg16-reduced/train_reduced.csv')

In [ ]:
class_train =  pd.read_csv('../input/targets-pneumo/Targets_train.csv')

In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(train_reduced, class_train, test_size=0.25, random_state=12)
rf = RandomForestClassifier()
rf.fit(X_F_train, y_C_train)
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

In [ ]:
corr_features = list()
corr_matrix = train_reduced.corr()
for i in range(len(corr_matrix .columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.3:
            colname = corr_matrix.columns[i]
            corr_features.append(str(colname))

In [ ]:
corr_features_ = np.asarray(corr_features)
corr_features_ = np.unique(corr_features_)
corr_features_

In [ ]:
vgg16_reduced_corr = train_reduced.drop(columns = corr_features_, axis=1)

In [ ]:
vgg16_reduced_corr

In [ ]:
vgg16_reduced_corr = vgg16_reduced_corr.drop(columns = ['Unnamed: 0'])

In [ ]:
vgg16_reduced_corr

In [ ]:
vgg16_reduced_corr.to_csv('vgg16_reduced_corr', sep=',', encoding='utf-8')

In [ ]:
class_train = class_train.drop(columns = ['Unnamed: 0']).T

In [ ]:
class_train.shape

In [ ]:
vgg16_reduced_corr.shape

In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(vgg16_reduced_corr, class_train, test_size=0.30, random_state=12)
rf = RandomForestClassifier()
rf.fit(X_F_train, y_C_train)
prediction = rf.predict(X_F_test)
class_pred = prediction.round()
class_true = y_C_test
print(classification_report(class_pred, class_true))

In [ ]:
prediction = rf.predict(vgg16_reduced_corr)
class_pred = prediction.round()
class_true = class_train
print(classification_report(class_pred, class_true))

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

rf_ = RandomForestClassifier() 
rfecv = RFECV(estimator=rf_, step=1, cv=5,scoring='recall_macro')
rfecv = rfecv.fit(X_F_train, y_C_train)

In [ ]:
print('Optimal number of features :', rfecv.n_features_)
print('Best features :', X_F_train.columns[rfecv.support_])

In [ ]:
df_best_features = df_vgg16.loc[:,[]]

In [ ]:
fetures_vgg16['F_VGG16'] = X_test_features

# Finally: Hybrid dataset 

In [ ]:
vgg16_reduced_corr

In [ ]:
df_reduced_corr

In [ ]:
df_final = pd.merge(df_reduced_corr, vgg16_reduced_corr, how = 'left', left_index = True, right_index = True)
df_final.to_csv('dataframe_king', sep=',', encoding='utf-8')

In [ ]:
df_final

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
predictors = scaler.fit_transform(df_final)

In [ ]:
predictors

In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier()
rf.fit(predictors, class_train)
scores = cross_val_score(rf, df_final, class_train, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
scores = cross_val_score(rf, df_final, class_train, cv=3, scoring='recall_macro')
print("%0.2f recall_macro with a standard deviation of %0.2f" % (scores.mean(), scores.std()))



#X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(df_final, class_train, test_size=0.30, random_state=12)
#rf = RandomForestClassifier()

#rf.fit(df_final, class_train)
#prediction = rf.predict(df_final)
#class_pred = prediction.round()
#class_true = y_C_test
#print(classification_report(class_pred, class_train))

In [ ]:
X_F_train, X_F_test, y_C_train, y_C_test = train_test_split(predictors, class_train, test_size=0.30, random_state=12)

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(verbose = True, max_iter = 1000, tol = 0.000010, solver='adam', hidden_layer_sizes=(100), activation = 'relu', batch_size=200, learning_rate_init=0.001)
MLP.fit(predictors, class_train)

In [ ]:
scores = cross_val_score(MLP, predictors, class_train, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
scores = cross_val_score(MLP, predictors, class_train, cv=3, scoring='recall_macro')
print("%0.2f recall_macro with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
X_F_train.shape

In [ ]:
classifier = models.Sequential()
classifier.add(layers.Dense(128, activation='relu',input_dim=245))
classifier.add(layers.Dropout(0.3))
classifier.add(layers.Dense(64, activation='relu'))
classifier.add(layers.Dropout(0.2))
classifier.add(layers.Dense(1, activation='sigmoid'))
classifier.summary()
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
result_class = classifier.fit(X_F_train, y_C_train, validation_data=(X_F_test, y_C_test),batch_size=8, epochs=100)